In [3]:
import pandas as pd

df = pd.read_csv('./df_50.csv')
#print(df)

In [20]:
#len(set(df['colorID']))

50

In [4]:
import numpy as np
import time
import cv2
import urllib

imgGraySmall = []

for m in range(len(df['imgURL'])):
    res = urllib.request.urlopen(df['imgURL'][m])
    i = np.asarray(bytearray(res.read()), dtype="uint8")
    imgGray = cv2.imdecode(i, cv2.IMREAD_GRAYSCALE)
    imgResize = cv2.resize(imgGray, (100, 100), interpolation=cv2.INTER_AREA)
    imgGraySmall.insert(m,imgResize)
    time.sleep(1)

In [ ]:
#print (imgGraySmall)
#print (len(imgGraySmall))

#for a in range():
#     cv2.imwrite("./img/{}.jpg".format(df.loc[a, "colorID"]), imgGraySmall[a])

In [2]:
#import cvzone
#from cvzone.SelfiSegmentationModule import SelfiSegmentation

#segmentor = SelfiSegmentation()
#img_Out = segmentor.removeBG(imgGraySmall[0], (255), threshold=0.99)

#https://github.com/google/mediapipe/issues/3849
#https://github.com/google/mediapipe/blob/master/docs/getting_started/python.md#building-mediapipe-python-package

ModuleNotFoundError: No module named 'mediapipe'

In [5]:
# 對圖像進行二值化，得到二值圖像
ret, thresh = cv2.threshold(imgGraySmall[0], 127, 255, cv2.THRESH_BINARY)

# 找到圖像中的輪廓
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 將輪廓繪製在新的圖像上，並去除背景
mask = np.zeros_like(imgGraySmall[0])
cv2.drawContours(mask, contours, -1, (255), -1)
result = cv2.bitwise_and(imgGraySmall[0], mask)

# 顯示結果
cv2.imshow('Result', result)
cv2.waitKey(0)

-1

In [22]:
#gray_cutoff = pd.DataFrame(imgGraySmall[0].flatten()).value_counts().index[0][0]

In [5]:
tfR = []
for n in range(len(imgGraySmall)): 
    imgGraySmall[n][ imgGraySmall[n] == np.argmax(np.bincount(imgGraySmall[n].flatten()))]= 255 #找出背景最多的數字，再指定成255
    img90 = cv2.rotate(imgGraySmall[n], cv2.ROTATE_90_CLOCKWISE)
    img180 = cv2.rotate(imgGraySmall[n], cv2.ROTATE_180)
    img270 = cv2.rotate(imgGraySmall[n], cv2.ROTATE_90_COUNTERCLOCKWISE)
    tfR.extend([imgGraySmall[n], img90, img180, img270])

tfF = []
for o in range(len(tfR)):
    imgUD = cv2.flip(tfR[o], 0)
    tfF.extend([tfR[o], imgUD])        

tfN = []
for p in range(len(tfF)):
    img_negative = 255-tfF[p]
    img_negative[ img_negative == np.argmax(np.bincount(img_negative.flatten()))]= 255
    tfN.extend([tfF[p], img_negative])

#print(len(tfR))
#print(len(tfF))
#print(len(tfN))


In [6]:
# for r in range(16):
#     cv2.imwrite("image{}.png".format(r), tfN[r])

In [7]:
flatlist = []
for q in tfN:
    flatlist.append(q.flatten())

In [ ]:
#flatlist

In [8]:
aaa = pd.DataFrame(flatlist).astype('uint8')

In [9]:
namelist = []
for r in df['productID']:
    for s in range(16):
        namelist.append(r)

In [47]:
#len(namelist)

800

In [10]:
aaa.index = namelist

In [11]:
#aaa

In [12]:
aaa.to_csv('transformation.csv', encoding = 'utf8')